# ALGOS DE RECO #

In [60]:
# on importe les differentes librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix

# on importe les différentes librairies surprise de scikit
from surprise import SVD
from surprise import dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans

# on importe notre bdd
avis = pd.read_csv("BDD/avis.csv", index_col="Unnamed: 0")
jeux = pd.read_csv("BDD/jeux.csv", index_col="Unnamed: 0")

In [29]:
jeux.head(1)

,titre,categories,description,full_title,Note,Nombre d'avis
0,Marvel United,NaN,\n Marvel United : Unis c...,Marvel United (2020),7.615385,13


In [30]:
avis.head(1)

,author,title_review,note,title,comment
0,Monsieur Guillaume,Voyages sur les ailes des papillons,8,Mariposas,"Lorsque le jeu est jeu, bon, réflexif, joli po..."


In [72]:
num_users  = avis["author"].nunique()
num_items  = avis["title"].nunique()

print(f"Il y'a {num_users} auteurs et {num_items} jeux")

sparsity = (len(avis) / (num_users*num_items))*100
print(f"Notre matrice a une sparsité de {sparsity:.2f}% ")

matrice_sparse = csr_matrix(avis['note'])
matrice_sparse



Il y'a 2459 auteurs et 3337 jeux
Notre matrice a une sparsité de 1.44% 


<1x118107 sparse matrix of type '<class 'numpy.int64'>'
	with 117772 stored elements in Compressed Sparse Row format>

## SVD ##

In [75]:
algo = SVD()

reader = Reader(line_format='user item rating', rating_scale=(0,10))

class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, jeux, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(avis['author'], avis['title'], avis['note'])]
        self.reader=reader

data = MyDataset(jeux, reader)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8719  1.8634  1.8485  1.8654  1.8588  1.8616  0.0078  
MAE (testset)     1.4323  1.4293  1.4209  1.4381  1.4276  1.4297  0.0056  
Fit time          0.79    0.77    0.73    0.78    0.82    0.78    0.03    
Test time         0.07    0.07    0.07    0.08    0.08    0.07    0.00    


{'test_rmse': array([1.8718834 , 1.86339389, 1.84850026, 1.86538011, 1.85877392]),
 'test_mae': array([1.4323081 , 1.42934727, 1.42094964, 1.43814575, 1.4276176 ]),
 'fit_time': (0.7942562103271484,
  0.7691311836242676,
  0.734872579574585,
  0.782677173614502,
  0.8249273300170898),
 'test_time': (0.07341575622558594,
  0.07102680206298828,
  0.06951332092285156,
  0.08241033554077148,
  0.07611227035522461)}

## k-NN ##

In [76]:
algo = KNNBasic()

reader = Reader(line_format='user item rating', rating_scale=(0,10))

class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, jeux, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(avis['author'], avis['title'], avis['note'])]
        self.reader=reader

data = MyDataset(jeux, reader)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.9141  1.8932  1.9018  1.9253  1.9285  1.9126  0.0135  
MAE (testset)     1.4839  1.4695  1.4721  1.4881  1.4952  1.4818  0.0097  
Fit time          0.39    0.39    0.39    0.39    0.39    0.39    0.00    
Test time         0.95    1.05    0.91    0.91    0.92    0.95    0.05    


{'test_rmse': array([1.91406615, 1.89318446, 1.90181797, 1.9252659 , 1.92847951]),
 'test_mae': array([1.483927  , 1.46954655, 1.47214029, 1.48808983, 1.49517182]),
 'fit_time': (0.38590312004089355,
  0.3878962993621826,
  0.3915421962738037,
  0.3913722038269043,
  0.39187026023864746),
 'test_time': (0.9506194591522217,
  1.0524318218231201,
  0.9143631458282471,
  0.9110374450683594,
  0.9218132495880127)}